## Create netCDF for complex DSGs

The CF Conventions provide recommendations on netCDF structure for two types of complex DSGs, i.e. **time series of profiles** and **trajectory of profiles**.


### Time Series of Profiles

* Time series of profiles at a single station [(H.5.2)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_time_series_of_profiles_at_a_single_station): There is only one stations in a dataset; for this station, multiple profiles were measured at multiple times.

* Multidimensional array representations of time series profiles [(H.5.1)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_multidimensional_array_representations_of_time_series_profiles): There are multiple stations in a dataset; for each station, one or more profiles were measured at multiple times; and there are the same number of profiles (i.e. time points for all time series), and the same number of vertical levels for every profile, whereas the time values and the vertical level values don't have to be the same. *For the situation where all time series share common time values, and all profiles share common vertical level values*, we can refer to the example H.17 in Appendix H.5.1.

* Ragged array representation of time series profiles [(H.5.3)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_ragged_array_representation_of_time_series_profiles): There are multiple stations, the number of profiles and vertical levels for each station varies.


### Trajectory of Profiles

* Profiles along a single trajectory [(H.6.2)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_profiles_along_a_single_trajectory)

* Multidimensional array representation of trajectory profiles [(H.6.1)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_multidimensional_array_representation_of_trajectory_profiles)

* Ragged array representation of trajectory profiles [(H.6.3)](https://cfconventions.org/Data/cf-conventions/cf-conventions-1.11/cf-conventions.html#_ragged_array_representation_of_trajectory_profiles)

```{note}
It's not always a good choice to pack all data into one single netCDF file. If thins get too complicated, it's often a better choice to separate the data by stations and put them into different netCDF files.
```

In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import xarray as xr
import cftime
from datetime import datetime

In [3]:
# List available datasets. Please change it to your file path.
os.chdir('/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data')
trj_files = glob(os.path.join(os.getcwd(), "dsg_trjProfile", "*.nc"))
trj_files

['/Users/icdc/Documents/NFDI/Kemeng/cfbook/src/data/dsg_trjProfile/Globec_bottle_data_2002.nc']